# Trabajo Práctico 2: KNN

La idea de este modelo es dado una nueva instancia devuelve la clase mayoritaria dada una cantidad de vecinos fijo(K). Para buscar los mejores hiperparametro utilizaremos GridSearch con un k-fold de 10.

+ n_neighbors: La cantidad de vecinos mas cercanos que debe ser tomados en cuanta para predecir la nueva instancia.

+ weights : La importancia que se le debe dar a cada vecino. Si es 'distance' su importancia es la inversa de su disancia a la instancia a predecir y si es 'uniform' todas valen lo mismo.

+ algorithm : Utilizado para calcular los vecinos más cercanos

+ p : Cuando p = 1, esto equivale a usar manhattan_distance (l1) y euclidean_distance (l2) para p = 2.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from preprocesamiento import feature_engineering
from preprocesamiento import obtenerDFHoldout, obtenerDFTraining
from preprocesamiento import preprocesar_data_frame

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
RANDOM_STATE = 19 * 103785

In [2]:
from preprocesamiento import metricas
from preprocesamiento import prepros_dummies
from preprocesamiento import preprocesar_standar_scaler

Inicialmente preprocesamos el dataset seleccionamos las variables más relevantes y aplicamos a "ganacia_declarada_en_bolsa" una estandarización ya que es una variable con mucha varianza.

In [3]:
df = obtenerDFTraining()
(X,y) = preprocesar_data_frame(df)
X_prepos = prepros_dummies(X[['ganancia_perdida_declarada_bolsa_argentina','trabajo',
                              'estado_marital','genero',"anios_estudiados","edad"]])
X_prepos = preprocesar_standar_scaler(X_prepos)

X_train, X_test, y_train, y_test = train_test_split(X_prepos, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
parametros = {'n_neighbors' : range(5,100,5),'weights':['uniform','distance'],
              'algorithm':['auto','kd_tree','brute'],'p':[1,2]}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parametros,n_jobs =-1,verbose = 4, scoring = 'roc_auc', cv= 10)

clf.fit(X_train,y_train)
print(classification_report(y_test,clf.predict(X_test)))
clf.best_params_

Fitting 10 folds for each of 228 candidates, totalling 2280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  86 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 209 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 380 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 601 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 870 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 1189 tasks      | elapsed: 18.5min


Como podemos ver nuestro primer modelo clasifica correctamente la mitad de las personas con poder adquisitivo alto, sin embargo logra separar bastante bien a las personas con bajo poder adquisitivo, clasificando la gran mayoría correctamente.

In [ ]:
knn_todos_los_features = KNeighborsClassifier(algorithm = 'auto', n_neighbors = 40, p = 1, weights = 'uniform')

knn_todos_los_features.fit(X_train,y_train)
metricas(y_test,knn_todos_los_features.predict(X_test),X_test,knn_todos_los_features)

## Otro preprocesamiento (PCA)

El objetivo es utilizar una herramienta de reducción de dimensionalidad para intentar mejorar nuestros resultados. Inicialmente hacemos PCA utilizando todo el dataset y luego seleccionando las variables, que por lo visto en la parte uno, son más relevantes para la clasificación.

In [ ]:
from preprocesamiento import preprocesar_df_min_max_scaler
from preprocesamiento import preprocesar_df_pca

df = obtenerDFTraining()
X, y = preprocesar_data_frame(df)
X = preprocesar_df_min_max_scaler(X)
X = preprocesar_df_pca(X, 30)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
parametros = {'n_neighbors' : range(5,100,5),'weights':['uniform','distance'],
              'algorithm':['auto','kd_tree','brute'],'p':[1,2]}

knn_pca_todos_los_features = KNeighborsClassifier()
clf = GridSearchCV(knn, parametros,n_jobs =-1,verbose = 4, scoring = 'roc_auc', cv= 10)

knn_pca_todos_los_features.fit(x_train,y_train)
print(classification_report(y_test,knn_pca_todos_los_features.predict(x_test)))
knn_pca_todos_los_features.best_params_

In [ ]:
metricas(y_test,knn_pca_todos_los_features.predict(x_test),x_test,knn_pca_todos_los_features)

Como podemos ver nuestro nuevo modelo pierde precisión pero aumento su recall para clasificar los unos.

Ahora por último haremos PCA pero sobre un dataset reducido con los mismos features que en el primer modelo.

In [ ]:
df = obtenerDFTraining()
X, y = preprocesar_data_frame(df)
X = preprocesar_df_min_max_scaler(X[['ganancia_perdida_declarada_bolsa_argentina','trabajo',
                          'estado_marital','genero',"anios_estudiados","tiene_alto_valor_adquisitivo"]])
X = preprocesar_df_pca(X, 18)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

parametros = {'n_neighbors' : range(5,100,5),'weights':['uniform','distance'],
              'algorithm':['auto','kd_tree','brute'],'p':[1,2]}

knn_pca_algunos_features = KNeighborsClassifier()
clf = GridSearchCV(knn, parametros,n_jobs =-1,verbose = 4, scoring = 'roc_auc', cv= 10)

clf.fit(x_train,y_train)
print(classification_report(y_test,knn_pca_algunos_features.predict(x_test)))
knn_pca_algunos_features.best_params_

Como podemos notar mejorar las métricas con respecto a los demás posibles modelos. De modo que decidimos presentarlo como modelo final de KNN.

In [ ]:
metricas(y_test,knn_pca_algunos_features.predict(x_test),x_test,knn_pca_algunos_features)

### Prediccion sobre el Holdout

In [ ]:
holdout = obtenerDFHoldout()

ids = np.array(holdout['id'])
holdout = preprocesar_df_min_max_scaler(holdout[['ganancia_perdida_declarada_bolsa_argentina','trabajo',
                          'estado_marital','genero',"anios_estudiados","tiene_alto_valor_adquisitivo"]])
holdout = preprocesar_df_pca(holdout,18)

predicciones_holdout = knn_pca_algunos_features.predict(holdout)
escribir_predicciones_a_archivo(predicciones_holdout,"KNN",ids)